In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

import pickle

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/MLOps-Zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1719736280654, experiment_id='1', last_update_time=1719736280654, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

Link to download the data: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [3]:
def read_datafram(filname):
    df = pd.read_parquet(filname)

    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
train_df = read_datafram('data/green_tripdata_2021-01.parquet')
val_df = read_datafram('data/green_tripdata_2021-02.parquet')

In [5]:
train_df['PU_DO'] = train_df['PULocationID'] + '_' + train_df['DOLocationID']
val_df['PU_DO'] = val_df['PULocationID'] + '_' + val_df['DOLocationID']

In [6]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = train_df[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
y_train = train_df['duration'].values

val_dicts = val_df[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = val_df['duration'].values

In [8]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mse = mean_squared_error(y_val, y_pred, squared=False)

print('MSE: ', mse)

MSE:  7.758715209663881


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:
with mlflow.start_run():
  
  mlflow.set_tag('developer', 'esmail')
  
  mlflow.log_param('train-data-path', 'data/green_tripdata_2021-01.parquet')
  mlflow.log_param('test-data-path', 'data/green_tripdata_2021-02.parquet')
  
  alpha = 0.01
  mlflow.log_param('alpha', alpha)
  
  ls = Lasso(alpha=alpha)
  ls.fit(X_train, y_train)

  y_pred = ls.predict(X_val)

  rmse = mean_squared_error(y_val, y_pred, squared=False)

  print('RMSE: ', rmse)
  mlflow.log_metric('RMSE', rmse)

RMSE:  11.167275941179728


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [11]:
re = Ridge(alpha=1)
re.fit(X_train, y_train)

y_pred = re.predict(X_val)

mse = mean_squared_error(y_val, y_pred, squared=False)

print('MSE: ', mse)

MSE:  7.703735141467915


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [115]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, re), f_out)

In [7]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [8]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [9]:
def objective(params):

  with mlflow.start_run():
    mlflow.set_tag('model', 'xgboost')
    mlflow.log_params(params)
    booster = xgb.train(
      params=params,
      dtrain=train,
      num_boost_round=500,
      evals=[(valid, 'validation')],
      early_stopping_rounds = 50
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)
  
  return {'loss': rmse, 'status': STATUS_OK}
  

In [26]:
search_space = {
  'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
  'learning_rate': hp.loguniform('learing_rate', -3, 0),
  'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
  'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
  'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
  'objective': 'reg:linear',
  'seed': 42
}

best_result = fmin(
  fn=objective,
  space=search_space,
  algo=tpe.suggest,
  max_evals=5,
  trials=Trials()
)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:19:32] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.86455                          
[1]	validation-rmse:6.85434                          
[2]	validation-rmse:6.61888                          
[3]	validation-rmse:6.54617                          
[4]	validation-rmse:6.51650                          
[5]	validation-rmse:6.50025                          
[6]	validation-rmse:6.49052                          
[7]	validation-rmse:6.48566                          
[8]	validation-rmse:6.47833                          
[9]	validation-rmse:6.46988                          
[10]	validation-rmse:6.46688                         
[11]	validation-rmse:6.46104                         
[12]	validation-rmse:6.45564                         
[13]	validation-rmse:6.44934                         
[14]	validation-rmse:6.44491                         
[15]	validation-rmse:6.44248                         
[16]	validation-rmse:6.43785                         
[17]	validation-rmse:6.43433                         
[18]	validation-rmse:6.43171

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:20:13] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.62109                                                  
[1]	validation-rmse:11.08515                                                  
[2]	validation-rmse:10.60065                                                  
[3]	validation-rmse:10.16403                                                  
[4]	validation-rmse:9.77072                                                   
[5]	validation-rmse:9.41642                                                   
[6]	validation-rmse:9.09914                                                   
[7]	validation-rmse:8.81535                                                   
[8]	validation-rmse:8.56210                                                   
[9]	validation-rmse:8.33598                                                   
[10]	validation-rmse:8.13502                                                  
[11]	validation-rmse:7.95613                                                  
[12]	validation-rmse:7.79728                        

...

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:23:04] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.27779                                                   
[1]	validation-rmse:8.97453                                                    
[2]	validation-rmse:8.12214                                                    
[3]	validation-rmse:7.57397                                                    
[4]	validation-rmse:7.22785                                                    
[5]	validation-rmse:7.00929                                                    
[6]	validation-rmse:6.87062                                                    
[7]	validation-rmse:6.77987                                                    
[8]	validation-rmse:6.71722                                                    
[9]	validation-rmse:6.67479                                                    
[10]	validation-rmse:6.64455                                                   
[11]	validation-rmse:6.62333                                                   
[12]	validation-rmse:6.60690            

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:24:01] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.76068                                                  
[1]	validation-rmse:11.34216                                                  
[2]	validation-rmse:10.95323                                                  
[3]	validation-rmse:10.59381                                                  
[4]	validation-rmse:10.26220                                                  
[5]	validation-rmse:9.95729                                                   
[6]	validation-rmse:9.67347                                                   
[7]	validation-rmse:9.41474                                                   
[8]	validation-rmse:9.17477                                                   
[9]	validation-rmse:8.95660                                                   
[10]	validation-rmse:8.75698                                                  
[11]	validation-rmse:8.57156                                                  
[12]	validation-rmse:8.40150                        

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [10]:
params = {
  'learning_rate': 0.2045786926241198,
  'max_depth': 17,
  'min_child_weight' : 1.2401309990275323,
  'objective' : 'reg:linear',
  'reg_alpha': 0.28563802598385397,
  'reg_lambda': 0.00428863291075846,
  'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
      params=params,
      dtrain=train,
      num_boost_round=500,
      evals=[(valid, 'validation')],
      early_stopping_rounds = 50
    )

2024/06/30 09:40:38 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2024/06/30 09:40:39 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '73c05e66edd54bd3a7026d45ce3e1b81', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:40:42] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.62212
[1]	validation-rmse:9.45609
[2]	validation-rmse:8.61406
[3]	validation-rmse:8.02316
[4]	validation-rmse:7.60764
[5]	validation-rmse:7.31540
[6]	validation-rmse:7.11566
[7]	validation-rmse:6.97634
[8]	validation-rmse:6.87754
[9]	validation-rmse:6.80639
[10]	validation-rmse:6.75525
[11]	validation-rmse:6.71833
[12]	validation-rmse:6.68897
[13]	validation-rmse:6.66945
[14]	validation-rmse:6.65242
[15]	validation-rmse:6.64038
[16]	validation-rmse:6.62927
[17]	validation-rmse:6.62193
[18]	validation-rmse:6.61513
[19]	validation-rmse:6.60958
[20]	validation-rmse:6.60683
[21]	validation-rmse:6.60397
[22]	validation-rmse:6.60275
[23]	validation-rmse:6.60180
[24]	validation-rmse:6.59976
[25]	validation-rmse:6.59748
[26]	validation-rmse:6.59494
[27]	validation-rmse:6.59271
[28]	validation-rmse:6.59132
[29]	validation-rmse:6.59004
[30]	validation-rmse:6.58766
[31]	validation-rmse:6.58506
[32]	validation-rmse:6.58362
[33]	validation-rmse:6.58252
[34]	validation-rmse:6.

2024/06/30 09:41:57 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/06/30 09:41:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [09:41:57] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
